#Scatter Plots

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations


files = {
    "O-Physico": "/content/strct_metrics_diamond.csv",
    "N-Physico": "/content/strct_metrics_mmseqs.csv",
    "ESMFold": "/content/strct_metrics_ESM.csv",
    "Baseline": "/content/strct_metrics_SingleAF.csv",
    "jackhmmer": "/content/merged_finalhh_with_unified.csv"
}

key_col = "unified_target"
group_col = "unified_dataset"
metrics = ["rmsd", "tm_score_chain1_norm", "Maxsub", "GDT_TS", "GDT_TA"]

names = {
    "rmsd": "RMSD",
    "tm_score_chain1_norm": "TM-score",
    "Maxsub": "MaxSub",
    "GDT_TS": "GDT-TS",
    "GDT_TA": "GDT-HA",
}


dfs = {}
for label, path in files.items():
    df = pd.read_csv(path)
    df.columns = [c.strip() for c in df.columns]
    dfs[label] = df


needed = {key_col, group_col, *metrics}
for label, df in dfs.items():
    miss = needed - set(df.columns)
    if miss:
        raise ValueError(f"{label} missing columns: {miss}")

all_groups = sorted(
    set().union(*[set(df[group_col].dropna().unique()) for df in dfs.values()])
)
palette = sns.color_palette(n_colors=len(all_groups))
palette_map = dict(zip(all_groups, palette))

def plot_pairwise_scatter(dfA, dfB, labelA, labelB, metric):
    common_df = pd.merge(
        dfA[[key_col, group_col, metric]],
        dfB[[key_col, group_col, metric]],
        on=[key_col, group_col],
        how="inner",
        suffixes=("_a", "_b")
    )

    xcol = f"{metric}_a"
    ycol = f"{metric}_b"

    finite_vals = pd.concat([common_df[xcol], common_df[ycol]], axis=0)\
        .replace([np.inf, -np.inf], np.nan).dropna()

    plt.figure(figsize=(6, 6))
    sns.scatterplot(
        data=common_df,
        x=xcol, y=ycol,
        hue=group_col,
        hue_order=all_groups,
        palette=palette_map,
        s=35,
        alpha=0.85
    )

    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')


    if len(finite_vals) > 0:
        mn, mx = float(finite_vals.min()), float(finite_vals.max())
        plt.plot([mn, mx], [mn, mx], linestyle="--", color="black", label="y=x")

    plt.xlabel(labelA)
    plt.ylabel(labelB)
    plt.title(f"{names.get(metric, metric)}  |  {labelA} vs {labelB}")
    plt.legend( bbox_to_anchor=(1.02, 1), loc="upper left")
    plt.tight_layout()
    plt.show()

labels = list(dfs.keys())
for metric in metrics:
    for labelA, labelB in combinations(labels, 2):
        plot_pairwise_scatter(dfs[labelA], dfs[labelB], labelA, labelB, metric)


#Violin Plots

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

names = {
    "rmsd": "RMSD",
    "tm_score_chain1_norm": "TM-score",
    "Maxsub": "MaxSub",
    "GDT_TS": "GDT-TS",
    "GDT_TA": "GDT-HA",
}

files = {
    "O-Physico": "/content/strct_metrics_diamond.csv",
    "N-Physico": "/content/strct_metrics_mmseqs.csv",
    "ESMFold": "/content/strct_metrics_ESM.csv",
    "Baseline": "/content/strct_metrics_SingleAF.csv",
    "jackhmmer": "/content/merged_finalhh_with_unified.csv"
}


base_dir = ""

group_col = "unified_dataset"
metrics = list(names.keys())


all_rows = []
for method_label, fname in files.items():
    df = pd.read_csv(f"{base_dir}/{fname}")
    df.columns = [c.strip() for c in df.columns]

    keep_cols = [group_col] + metrics
    missing = set(keep_cols) - set(df.columns)
    if missing:
        raise ValueError(f"{fname} missing columns: {missing}")

    dff = df[keep_cols].copy()
    dff["method"] = method_label
    all_rows.append(dff)

big = pd.concat(all_rows, ignore_index=True)
method_order = ["O-Physico", "N-Physico", "Baseline", "ESMFold","jackhmmer"]

method_palette = dict(zip(method_order, sns.color_palette("Set1", n_colors=len(method_order))))


groups = big[group_col].dropna().unique().tolist()
groups = sorted(groups)

for g in groups:
    sub_g = big[big[group_col] == g].copy()

    for metric in metrics:
        plot_df = sub_g[["method", metric]].rename(columns={metric: "value"}).dropna()

        fig, ax = plt.subplots(figsize=(10, 4.8))

        sns.violinplot(
            data=plot_df,
            x="method",
            y="value",
            order=method_order,
            palette=method_palette,   # رنگ بر اساس ابزار
            #cut=0,
            inner="box",
            linewidth=1,
            ax=ax,
        )

        ax.set_title(f"{names[metric]} Distribution | {g}")
        ax.set_xlabel("")
        ax.set_ylabel(names[metric])


        fig.tight_layout()
        plt.show()


#Pairwise Test and Forest Plots

In [ ]:

"""
Pairwise comparison of 4 structure-metric CSVs (joined on unified_target),
split into 4 unified_dataset categories, grouped per category+target, then:
- Wilcoxon signed-rank (paired)
- Cohen's d (paired; dz = mean(diff)/std(diff))
- 95% CI plot (bootstrap CI of mean paired difference)

Inputs are the 4 files you uploaded. Outputs:
- pairwise_stats.csv
- plots/*.png  (forest plots per category × metric)

Requirements:
pip install pandas numpy scipy matplotlib
"""

import os
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import wilcoxon

# ----------------------------
# Labels (as you requested)
# ----------------------------
files = {
    "O-Physico": "/content/strct_metrics_diamond.csv",
    "N-Physico": "/content/strct_metrics_mmseqs.csv",
    "ESMFold": "/content/strct_metrics_ESM.csv",
    "Baseline": "/content/strct_metrics_SingleAF.csv",
    "jackhmmer": "/content/merged_finalhh_with_unified.csv"
}

metric_labels = {
    "rmsd": "RMSD",
    "tm_score_chain1_norm": "TM-score",
    "Maxsub": "MaxSub",
    "GDT_TS": "GDT-TS",
    "GDT_TA": "GDT-HA",  # NOTE: you wrote GDT-HA for GDT_TA; keeping exactly that label.
}

METRICS = list(metric_labels.keys())

# ----------------------------
# Paths (mounted in this environment)
# If you run locally, replace these with your real paths.
# ----------------------------
mounted_paths = {
    "O-Physico": "/content/strct_metrics_diamond.csv",
    "N-Physico": "/content/strct_metrics_mmseqs.csv",
    "ESMFold": "/content/strct_metrics_ESM.csv",
    "Baseline": "/content/strct_metrics_SingleAF.csv",
    "jackhmmer": "/content/merged_finalhh_with_unified.csv"
}

JOIN_KEY = "unified_target"
CAT_COL = "unified_dataset"  # 4 دسته

OUT_DIR = "outputs"
PLOT_DIR = os.path.join(OUT_DIR, "plots")
os.makedirs(PLOT_DIR, exist_ok=True)

# ----------------------------
# Helpers
# ----------------------------
def drop_unnamed(df: pd.DataFrame) -> pd.DataFrame:
    return df.loc[:, ~df.columns.astype(str).str.startswith("Unnamed:")].copy()

def paired_cohens_dz(diff: np.ndarray) -> float:
    """Cohen's d for paired samples (dz)."""
    diff = np.asarray(diff, dtype=float)
    diff = diff[np.isfinite(diff)]
    if diff.size < 2:
        return np.nan
    sd = diff.std(ddof=1)
    if sd == 0:
        return np.nan
    return diff.mean() / sd

def bootstrap_mean_ci(diff: np.ndarray, n_boot: int = 5000, ci: float = 0.95, seed: int = 123) -> tuple[float, float]:
    """Bootstrap CI for mean of paired differences."""
    rng = np.random.default_rng(seed)
    diff = np.asarray(diff, dtype=float)
    diff = diff[np.isfinite(diff)]
    n = diff.size
    if n == 0:
        return (np.nan, np.nan)
    if n == 1:
        return (diff[0], diff[0])
    boots = rng.choice(diff, size=(n_boot, n), replace=True).mean(axis=1)
    alpha = (1 - ci) / 2
    lo = np.quantile(boots, alpha)
    hi = np.quantile(boots, 1 - alpha)
    return (lo, hi)

def safe_wilcoxon(x: np.ndarray, y: np.ndarray):
    """
    Wilcoxon signed-rank test on paired vectors x,y
    Returns (stat, p).
    Handles edge cases where all diffs are zero / too small.
    """
    x = np.asarray(x, dtype=float)
    y = np.asarray(y, dtype=float)
    mask = np.isfinite(x) & np.isfinite(y)
    x = x[mask]
    y = y[mask]
    if x.size < 2:
        return (np.nan, np.nan)

    # If all differences are zero, wilcoxon can error depending on scipy version.
    d = x - y
    if np.allclose(d, 0, equal_nan=False):
        return (0.0, 1.0)

    try:
        stat, p = wilcoxon(x, y, zero_method="wilcox", correction=False, alternative="two-sided", mode="auto")
        return (float(stat), float(p))
    except Exception:
        # fallback: drop exact-zero diffs
        nz = ~np.isclose(d, 0)
        if nz.sum() < 2:
            return (np.nan, np.nan)
        stat, p = wilcoxon(x[nz], y[nz], zero_method="wilcox", correction=False, alternative="two-sided", mode="auto")
        return (float(stat), float(p))

def forest_plot(df_plot: pd.DataFrame, title: str, out_path: str):
    """
    df_plot columns required:
      comparison, mean_diff, ci_low, ci_high
    """
    df_plot = df_plot.copy().sort_values("comparison", ascending=True)

    y = np.arange(len(df_plot))
    mean = df_plot["mean_diff"].to_numpy()
    lo = df_plot["ci_low"].to_numpy()
    hi = df_plot["ci_high"].to_numpy()
    xerr = np.vstack([mean - lo, hi - mean])

    plt.figure(figsize=(10, max(3.5, 0.55 * len(df_plot) + 1.5)))

    plt.errorbar(
        mean, y, xerr=xerr,
        fmt="o",
        capsize=4,
        color="black",
        ecolor="black",
        markeredgecolor="black",
        markerfacecolor="black",
    )


    plt.axvline(0, linewidth=1.5, color="red")

    plt.yticks(y, df_plot["comparison"].tolist())
    plt.xlabel("Mean paired difference (A - B) with 95% CI")
    plt.title(title)
    plt.tight_layout()
    plt.savefig(out_path, dpi=200)
    plt.close()


# ----------------------------
# Load + prepare
# ----------------------------
dfs = {}
for model, path in mounted_paths.items():
    df = pd.read_csv(path)
    df = drop_unnamed(df)
    missing = [c for c in [JOIN_KEY, CAT_COL] + METRICS if c not in df.columns]
    if missing:
        raise ValueError(f"[{model}] Missing columns: {missing}\nFound: {df.columns.tolist()}")
    dfs[model] = df

# Group first (per category + target) then mean (in case duplicates exist)
gdfs = {}
for model, df in dfs.items():
    g = (
        df.groupby([CAT_COL, JOIN_KEY], as_index=False)[METRICS]
          .mean(numeric_only=True)
    )
    gdfs[model] = g

# Merge all models on [unified_dataset, unified_target]
merged = None
for model, df in gdfs.items():
    tmp = df.rename(columns={m: f"{model}__{m}" for m in METRICS})
    if merged is None:
        merged = tmp
    else:
        merged = merged.merge(tmp, on=[CAT_COL, JOIN_KEY], how="inner")

if merged is None or merged.empty:
    raise RuntimeError("Merged dataset is empty. Check join keys / file contents.")

# ----------------------------
# Pairwise stats
# ----------------------------
models = list(files.keys())
pairs = list(itertools.combinations(models, 2))
categories = sorted(merged[CAT_COL].dropna().unique().tolist())

rows = []
for cat in categories:
    sub = merged.loc[merged[CAT_COL] == cat].copy()

    for metric in METRICS:
        for a, b in pairs:
            col_a = f"{a}__{metric}"
            col_b = f"{b}__{metric}"

            xa = sub[col_a].to_numpy(dtype=float)
            xb = sub[col_b].to_numpy(dtype=float)

            mask = np.isfinite(xa) & np.isfinite(xb)
            xa = xa[mask]
            xb = xb[mask]

            if xa.size == 0:
                continue

            diff = xa - xb
            stat, p = safe_wilcoxon(xa, xb)
            d_dz = paired_cohens_dz(diff)
            mean_diff = float(np.nanmean(diff)) if diff.size else np.nan
            ci_low, ci_high = bootstrap_mean_ci(diff, n_boot=5000, ci=0.95, seed=123)

            rows.append({
                "unified_dataset": cat,
                "metric": metric,
                "metric_label": metric_labels[metric],
                "model_A": a,
                "model_B": b,
                "n_pairs": int(diff.size),
                "wilcoxon_stat": stat,
                "wilcoxon_p": p,
                "cohens_dz": d_dz,
                "mean_diff_A_minus_B": mean_diff,
                "ci95_low": ci_low,
                "ci95_high": ci_high,
            })

stats_df = pd.DataFrame(rows)
out_csv = os.path.join(OUT_DIR, "pairwise_stats.csv")
os.makedirs(OUT_DIR, exist_ok=True)
stats_df.to_csv(out_csv, index=False)

print(f"[OK] Saved stats: {out_csv}")
print(stats_df.head(10).to_string(index=False))

# ----------------------------
# Plots: per category × metric (forest plot across all pairwise comparisons)
# ----------------------------
for cat in categories:
    for metric in METRICS:
        sdf = stats_df[(stats_df["unified_dataset"] == cat) & (stats_df["metric"] == metric)].copy()
        if sdf.empty:
            continue

        sdf["comparison"] = sdf["model_A"] + " vs " + sdf["model_B"]
        df_plot = sdf[["comparison", "mean_diff_A_minus_B", "ci95_low", "ci95_high"]].rename(
            columns={
                "mean_diff_A_minus_B": "mean_diff",
                "ci95_low": "ci_low",
                "ci95_high": "ci_high",
            }
        )




        title = f"{cat} | {metric_labels[metric]} | Pairwise mean diff (A-B) with 95% CI"
        out_png = os.path.join(PLOT_DIR, f"{cat}__{metric}.png".replace("/", "_"))
        forest_plot(df_plot, title=title, out_path=out_png)

print(f"[OK] Saved plots in: {PLOT_DIR}")


#HeatMaps and VIF

In [ ]:
# @title
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LinearRegression

# === Step 0: Config ===
file_paths = [
    '/content/MSA_qualities_diamond.csv',
    '/content/MSA_qualities_mmseqs.csv',
]


names={"/content/MSA_qualities_diamond.csv":"O-Physico"
,"/content/MSA_qualities_mmseqs.csv":"N-Physico"}




msa_features = ['NEff_id62','MeanEntropy','GapFraction','AvgIdentityToQuery']


# === Step 1: Load & Feature Engineering ===
def load_and_process(file_path):
    df = pd.read_csv(file_path)
    df = df.dropna()
    df = df.dropna(axis=1, how='all')

    return df

# === Step 2: Collinearity Analysis ===

# 2.1 Correlation Matrix
def compute_correlation_matrices(df):
    pearson_corr = df[msa_features].corr(method='pearson')
    spearman_corr = df[msa_features].corr(method='spearman')
    return pearson_corr, spearman_corr

# 2.2 VIF Calculation
def calculate_vif(df):
    X = df[msa_features].copy()
    vif_data = pd.DataFrame()
    vif_data['feature'] = X.columns
    vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data



labels=["Normalized NEff","Mean Entropy","Gap Fraction","Avg. Identity"]

# === Step 4: Visualization ===
def plot_heatmap(corr_matrix, title):
    plt.figure(figsize=(8,6))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', square=True, xticklabels=labels,yticklabels=labels)
    plt.title(title)
    plt.tight_layout()
    plt.show()

def analyze_file(file_path):
    print(f"\n=== Analyzing: {file_path} ===")
    df = load_and_process(file_path)

    # Step 2: Correlation Matrices
    pearson_corr, spearman_corr = compute_correlation_matrices(df)
    print("\nPearson Correlation Matrix:")
    print(pearson_corr)
    print("\nSpearman Correlation Matrix:")
    print(spearman_corr)

    # Step 2: VIF
    vif_table = calculate_vif(df)
    print("\nVIF Table:")
    print(vif_table)


    # Step 4: Plots
    plot_heatmap(pearson_corr, f"Pearson Correlation Heatmap | {names[file_path]}")
    plot_heatmap(spearman_corr, f"Spearman Correlation Heatmap | {names[file_path]}")


    return pearson_corr, spearman_corr, vif_table

# Main Execution
for path in file_paths:
    analyze_file(path)

# === Interpretation Guide ===
print("""
Interpretation:
- Multicollinearity check:
    - Pearson r > 0.8 (or < -0.8) indicates strong correlation.
    - VIF > 5 suggests multicollinearity; consider removing the feature.

- Feature-Quality Association:
    - Higher absolute Spearman correlation with TM, GDT-TS, GDT-HA suggests stronger relevance.
    - Feature selection recommendation based on low VIF and high correlation.
""")


In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import spearmanr
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import LinearRegression

# === Step 0: Config ===
file_paths = [
    ['/content/MSA_qualities_diamond.csv',
    '/content/strct_metrics_diamond.csv' ],

    ['/content/MSA_qualities_mmseqs.csv',
    '/content/strct_metrics_mmseqs.csv']


]

msa_features = ['NEff_id62','MeanEntropy','GapFraction','AvgIdentityToQuery']
quality_scores = ['tm_score_chain1_norm', 'GDT_TS', 'GDT_TA']

# === Step 1: Load & Feature Engineering ===
def load_and_process(file_path):
    df1 = pd.read_csv(file_path[0])
    df2 = pd.read_csv(file_path[1])

    df1 = df1.dropna(axis=1, how='all')
    df2 = df2.dropna(axis=1, how='all')


    merged_df = pd.merge(
    df1,
    df2,
    on="unified_id",
    how="inner"
    )


    print(merged_df.head())
    print("Rows:", merged_df.shape[0])
    print("Columns:", merged_df.shape[1])

    return merged_df


# === Step 3: Correlation with Structure Quality ===
def correlation_with_quality(df):
    results = []
    for feature in msa_features:
        for score in quality_scores:
            coef, pval = spearmanr(df[feature], df[score])
            results.append({
                'Feature': feature,
                'Score': score,
                'Spearman Correlation': coef,
                'p-value': pval
            })
    return pd.DataFrame(results)




labels=["Normalized NEff","Mean Entropy","Gap Fraction","Avg. Identity"]

labels2=["TM-Score","GDT-TS","GDT-HA"]


# === Step 4: Visualization ===
def plot_heatmap(corr_matrix, title):
    plt.figure(figsize=(8,6))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f', square=True,xticklabels=labels2[::-1],yticklabels=labels[::-1])
    plt.title(title)
    plt.tight_layout()
    plt.show()

def analyze_file(file_path):
    print(f"\n=== Analyzing: {file_path} ===")
    df = load_and_process(file_path)


    # Step 3: Correlation with Structure Quality
    corr_quality_table = correlation_with_quality(df)
    print("\nSpearman Correlation with TM, GDT-TS, GDT-HA:")
    print(corr_quality_table)


    # Heatmap for MSA Features vs Quality
    pivot_table = corr_quality_table.pivot(index='Feature', columns='Score', values='Spearman Correlation')
    plot_heatmap(pivot_table, f"Spearman Correlation (MSA Features vs Quality) | {names[file_path[0]]}")

    return corr_quality_table

# Main Execution
for path in file_paths:
    analyze_file(path)



#Box Plots

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# -----------------------------
# فایل‌ها
# -----------------------------
files = [
    "MSA_qualities_diamond.csv",
    "MSA_qualities_mmseqs.csv",
]

# -----------------------------
# نام ستون‌ها (دقیقاً از فایل)
# -----------------------------
COL_UNIFIED = "unified_dataset"
COL_NEFF = "NEff_id62"

# رنگ ثابت برای unified_dataset ها (۳ تا)
palette = ["#1f77b4", "#ff7f0e", "#2ca02c"]

# -----------------------------
# خواندن و رسم
# -----------------------------
for path in files:
    df = pd.read_csv(path)
    file_label = os.path.splitext(os.path.basename(path))[0]

    df = df[[COL_UNIFIED, COL_NEFF]].dropna()

    unified_order = sorted(df[COL_UNIFIED].unique())

    data_for_box = [
        df.loc[df[COL_UNIFIED] == u, COL_NEFF].values
        for u in unified_order
    ]

    plt.figure(figsize=(7, 5))

    bp = plt.boxplot(
        data_for_box,
        labels=unified_order,
        patch_artist=True,   # برای رنگ دادن
        showfliers=False     # اگر outlierها رو می‌خوای True کن
    )

    # رنگ دادن به باکس‌ها
    for box, color in zip(bp["boxes"], palette):
        box.set_facecolor(color)
        box.set_edgecolor("black")
        box.set_alpha(0.85)

    # تنظیم ظاهر خطوط
    for k in ["medians", "whiskers", "caps"]:
        for item in bp[k]:
            item.set_color("black")
            item.set_linewidth(1.2)

    plt.ylabel(COL_NEFF)
    plt.xlabel(COL_UNIFIED)
    plt.title(f"{COL_NEFF} Boxplot — {file_label}")

    plt.grid(axis="y", alpha=0.25)
    plt.tight_layout()
    plt.show()


#